In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import psycopg2
import pandas as pd
import psycopg2.extras

In [5]:
DATA_DIR = "C:/Users/Hp/Downloads/"

fact_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name=0, engine='openpyxl')
trans_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name=1, engine='openpyxl')
item_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name=2, engine='openpyxl')
customer_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name=3, engine='openpyxl')
time_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name=4, engine='openpyxl')
store_dim = pd.read_excel(open(DATA_DIR + 'e-commerece_data.xlsx','rb'), sheet_name=5, engine='openpyxl')




In [6]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="postgres",
                                           user = "postgres",
                                           password = "123456",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection


In [7]:
con = PostgresConnection().getConnection()

successfully connected to database


# Q1. 

In [8]:
cur = con.cursor()

cur.execute('''SELECT s.division, SUM(t.total_price) 
              FROM ecomdb.fact_table t 
              JOIN ecomdb.store_dim s on s.store_key=t.store_key 
                JOIN ecomdb.time_dim tim on tim.time_key=t.time_key 
                WHERE tim.month='12'
                GROUP BY s.division 
                ORDER BY s.division''')
cur.fetchall()


[('BARISAL', 638101.0),
 ('CHITTAGONG', 1665138.0),
 ('DHAKA', 3440676.25),
 ('KHULNA', 952049.75),
 ('RAJSHAHI', 1041201.25),
 ('RANGPUR', 721057.5),
 ('SYLHET', 465256.75)]

In [9]:
cur = con.cursor()
select_stmt = "SELECT t.payment_key , t.trans_type, t.bank_name " \
              "FROM ecomdb.trans_dim t"
cur.execute(select_stmt)
records = cur.fetchall()
records

[('P001', 'cash', 'None'),
 ('P002', 'card', 'AB Bank Limited'),
 ('P003', 'card', 'Bangladesh Commerce Bank Limited'),
 ('P004', 'card', 'Bank Asia Limited'),
 ('P005', 'card', 'BRAC Bank Limited'),
 ('P006', 'card', 'Citizens Bank Limited'),
 ('P007', 'card', 'City Bank Limited'),
 ('P008', 'card', 'Community Bank Bangladesh Limited'),
 ('P009', 'card', 'Dhaka Bank Limited'),
 ('P010', 'card', 'Dutch-Bangla Bank Limited'),
 ('P011', 'card', 'Eastern Bank Limited'),
 ('P012', 'card', 'IFIC Bank Limited'),
 ('P013', 'card', 'Jamuna Bank Limited'),
 ('P014', 'card', 'Meghna Bank Limited'),
 ('P015', 'card', 'Mercantile Bank Limited'),
 ('P016', 'card', 'Midland Bank Limited'),
 ('P017', 'card', 'Modhumoti Bank Limited'),
 ('P018', 'card', 'Mutual Trust Bank Limited'),
 ('P019', 'card', 'National Bank Limited'),
 ('P020', 'card', 'National Credit & Commerce Bank Limited'),
 ('P021', 'card', 'NRB Bank Limited'),
 ('P022', 'card', 'NRB Commercial Bank Ltd'),
 ('P023', 'card', 'NRB Global B

# Q2

In [10]:
cur = con.cursor()

cur.execute('''SELECT c.name,  SUM(f.total_price) 
              FROM ecomdb.fact_table f 
              JOIN ecomdb.customer_dim c on c.customer_key= f.customer_key 
               JOIN ecomdb.trans_dim t on t.payment_key= f.payment_key 
               where t.trans_type = 'cash'
               GROUP BY c.name ''')

cur.fetchall()


# customer =>  customer dim
# bank/transaction(cash/online) => trans dim
# total price => fact table (sum)


[('-', 254.0),
 ('aafreen fatima', 518.0),
 ('aafrin', 332.0),
 ('aafrin @ dolly', 717.25),
 ('aaftaab husan', 496.5),
 ('aakanksha', 625.0),
 ('aaliya', 1074.0),
 ('aamina', 128.0),
 ('aamir', 176.25),
 ('aamir hussain', 245.0),
 ('aamod & pramod', 234.0),
 ('aanamika misra', 143.0),
 ('aanchal', 1655.5),
 ('aansi devi', 182.25),
 ('aanya @ china', 71.0),
 ('aaradhana @ priyanka', 12.0),
 ('aarifa bano', 112.0),
 ('aarifun bivi', 276.25),
 ('aarti', 14815.25),
 ('aarti @ kajal', 68.0),
 ('aarti @ noni', 99.5),
 ('aarti bunkar', 484.0),
 ('aarti devi', 107.5),
 ('aarti diwakar', 441.0),
 ('aarti pal', 132.0),
 ('aarti rajpoot', 280.0),
 ('aarti shaha', 243.0),
 ('aarti sharma', 175.0),
 ('aarti singh @ pooja', 421.0),
 ('aasa devi', 100.75),
 ('aasha', 997.0),
 ('aasha devi', 260.0),
 ('aasha rani', 96.0),
 ('aashi', 302.0),
 ('aashiya', 416.25),
 ('aashma', 292.0),
 ('aashu', 496.5),
 ('aasish  kumar', 146.25),
 ('aasma', 597.75),
 ('aasmin', 630.0),
 ('aastha thakur', 70.0),
 ('aasu'

# Q3

In [11]:
cur = con.cursor()
cur.execute(''' SELECT SUM(f.total_price) 
                FROM ecomdb.fact_table f 
                JOIN ecomdb.store_dim s on s.store_key= f.store_key 
                 JOIN ecomdb.item_dim i on i.item_key= f.item_key 
                 where s.district = 'BARISAL' and  i.item_name = 'Pepsi - 12 oz cans' ''')


cur.fetchall()


[(12982.5,)]

# Q4

In [12]:
cur = con.cursor()

cur.execute(''' SELECT SUM(f.total_price) 
                FROM ecomdb.fact_table f 
                JOIN ecomdb.time_dim t on t.time_key = f.time_key 
                 JOIN ecomdb.item_dim i on i.item_key= f.item_key 
                 where t.year = '2015' and  i.supplier_ = 'BIGSO AB' ''')

cur.fetchall()

[(1685691.75,)]

# Q5

In [13]:
cur = con.cursor()

cur.execute(''' SELECT SUM(f.total_price) 
                FROM ecomdb.fact_table f
                JOIN ecomdb.store_dim s on s.store_key= f.store_key 
                JOIN ecomdb.time_dim t on t.time_key = f.time_key
                where s.division = 'DHAKA' and  t.year = '2015' ''')

cur.fetchall()

[(5865078.75,)]

# Q6

In [14]:
cur = con.cursor()

cur.execute(''' select * from
( 
SELECT i.supplier_, i.item_name, SUM(f.quantity) as quantity_sales, row_number() over(partition by i.supplier_ order by SUM(f.quantity) desc) rowid 
FROM ecomdb.fact_table f 
JOIN ecomdb.item_dim i on i.item_key= f.item_key 
JOIN ecomdb.store_dim s on s.store_key= f.store_key  
group by  i.supplier_, i.item_name
order by i.supplier_, rowid, SUM(f.quantity) desc	
) x
where rowid <= 3
order by x.supplier_, rowid ''')


cur.fetchall()


[('BIGSO AB', 'Nat.Valley PeanutButter Protein Bars', 23958, 1),
 ('BIGSO AB', '100% Juice Box Variety 6.75 oz ', 23425, 2),
 ('BIGSO AB', 'Red Bull Sugar Free 8.4 oz', 23343, 3),
 ('Bolsius Boxmeer', 'A&W Root Beer - 12 oz cans', 23623, 1),
 ('Bolsius Boxmeer', 'Waterloo Sparkling Original 12oz', 23289, 2),
 ('Bolsius Boxmeer', "Reese's Pieces Peanutbutter Candy", 23273, 3),
 ('CHERRY GROUP CO.,LTD', 'Wonderful Pistachios (in shell)', 23454, 1),
 ('CHERRY GROUP CO.,LTD', 'Clorox Bleach Large Jugs', 23158, 2),
 ('CHERRY GROUP CO.,LTD', 'Topo Chico 20oz plastic', 23108, 3),
 ('CHROMADURLIN S.A.S', 'A&W Root Beer Diet - 12 oz cans', 23182, 1),
 ('CHROMADURLIN S.A.S', 'Pop-Tarts, Brown Sugar Cinnamon ', 23165, 2),
 ('CHROMADURLIN S.A.S', "Reese's Peanutbutter Cups 2/pk", 23160, 3),
 ('DENIMACH LTD', 'Gatorade Variety 20 oz ', 23474, 1),
 ('DENIMACH LTD', 'Large Trash Bags Black 33 gal', 23435, 2),
 ('DENIMACH LTD', 'Charmin Ultra Bath Tissue', 23357, 3),
 ('Friedola 1888 GmbH', 'Cascade G

# Q7

In [15]:
cur = con.cursor()

cur.execute(''' SELECT x.item_name 
FROM 
(SELECT i.item_name, SUM(f.total_price) 
FROM ecomdb.fact_table f 
JOIN ecomdb.item_dim i on i.item_key= f.item_key 
JOIN ecomdb.trans_dim t on t.payment_key= f.payment_key
JOIN ecomdb.time_dim td on td.time_key = f.time_key
where (t.trans_type = 'card' or t.trans_type = 'mobile') and td.day <='5'
group by i.item_name)x 
group by x.item_name  ''')


cur.fetchall()



[('100% Juice Box Variety 6.75 oz ',),
 ('A&W Root Beer - 12 oz cans',),
 ('A&W Root Beer Diet - 12 oz cans',),
 ('Advil 2 pill packets',),
 ('All Purpose Cleaner 409 Spray',),
 ('Altoids Small Packs Variety',),
 ('Apple Sauce Variety GoGO Squeeze 3.2oz',),
 ('AquaFina Purified Drinking Water 16oz',),
 ('Austin Cookie Cracker Variety',),
 ('Austin Toasty Crackers w/ PBtr',),
 ('Baked Chips Variety Mix Frito',),
 ("Barq's Root Beer - 12 oz cans",),
 ('Belvita Hard Biscuits Blueberry',),
 ('Belvita Hard Biscuits Cinnamon',),
 ('Belvita Protein Oats Soft Biscuit',),
 ('Blue Diamond Almonds Rstd, Sltd',),
 ('Bounty Paper Towels Select a Size ',),
 ('Breath Savers Peppermints 12/roll ',),
 ('Brisk Lemon Iced Tea - 12 oz cans',),
 ('Cascade Gel Packs Dishwasher',),
 ('Cascade Gel w/ Dawn Dishwasher',),
 ('Charmin Ultra Bath Tissue',),
 ('Cheese and Peanut Butter Crackers',),
 ("Cheetos Flamin' Hot 1 oz",),
 ('Cheez It Original Crackers 1.5oz',),
 ('Cheez It White Cheddar 1.5 oz',),
 ('Cherry

# Q8

In [16]:
cur = con.cursor()

cur.execute('''select * from
(
SELECT x.item_name, x.quarter, MIN(x.total_sold) as min_total_sold, row_number() over(partition by x.item_name order by MIN(x.total_sold) asc) rowid 
FROM
(
SELECT i.item_name, t.quarter, sum(f.total_price) as total_sold
FROM ecomdb.fact_table f 
JOIN ecomdb.item_dim i on i.item_key= f.item_key 
JOIN ecomdb.time_dim t on t.time_key = f.time_key
group by i.item_name, t.quarter
) x
group by x.item_name, x.quarter
order by x.item_name asc, rowid, MIN(x.total_sold) asc
) y
where rowid = 1
order by y.item_name, rowid''')

cur.fetchall()




[('100% Juice Box Variety 6.75 oz ', 'Q1', 85470.0, 1),
 ('A&W Root Beer - 12 oz cans', 'Q2', 65952.5, 1),
 ('A&W Root Beer Diet - 12 oz cans', 'Q4', 37725.75, 1),
 ('Advil 2 pill packets', 'Q1', 76342.0, 1),
 ('All Purpose Cleaner 409 Spray', 'Q1', 95850.0, 1),
 ('Altoids Small Packs Variety', 'Q3', 65592.0, 1),
 ('Apple Sauce Variety GoGO Squeeze 3.2oz', 'Q4', 78190.0, 1),
 ('AquaFina Purified Drinking Water 16oz', 'Q1', 47691.0, 1),
 ('Austin Cookie Cracker Variety', 'Q1', 78358.0, 1),
 ('Austin Toasty Crackers w/ PBtr', 'Q2', 73836.0, 1),
 ('Baked Chips Variety Mix Frito', 'Q2', 86896.0, 1),
 ("Barq's Root Beer - 12 oz cans", 'Q3', 37840.5, 1),
 ('Belvita Hard Biscuits Blueberry', 'Q4', 70005.0, 1),
 ('Belvita Hard Biscuits Cinnamon', 'Q1', 74100.0, 1),
 ('Belvita Protein Oats Soft Biscuit', 'Q3', 73024.0, 1),
 ('Blue Diamond Almonds Rstd, Sltd', 'Q4', 32844.0, 1),
 ('Bounty Paper Towels Select a Size ', 'Q4', 146562.0, 1),
 ('Breath Savers Peppermints 12/roll ', 'Q2', 87872.0, 1),

# Q9

In [17]:
cur = con.cursor()

cur.execute(''' SELECT i.item_name, s.division, sum(f.total_price)
                FROM ecomdb.fact_table f 
                JOIN ecomdb.item_dim i on i.item_key= f.item_key 
                JOIN ecomdb.store_dim s on s.store_key= f.store_key 
                 group by i.item_name, s.division
                order by i.item_name desc ''')

cur.fetchall()



[('Zoo Animal Cookies/Crackers', 'BARISAL', 24178.0),
 ('Zoo Animal Cookies/Crackers', 'CHITTAGONG', 55762.0),
 ('Zoo Animal Cookies/Crackers', 'DHAKA', 118286.0),
 ('Zoo Animal Cookies/Crackers', 'KHULNA', 35434.0),
 ('Zoo Animal Cookies/Crackers', 'RAJSHAHI', 37100.0),
 ('Zoo Animal Cookies/Crackers', 'RANGPUR', 25186.0),
 ('Zoo Animal Cookies/Crackers', 'SYLHET', 14728.0),
 ('Yoplait Original Yogurt Variety Pack ', 'BARISAL', 19980.0),
 ('Yoplait Original Yogurt Variety Pack ', 'CHITTAGONG', 53652.0),
 ('Yoplait Original Yogurt Variety Pack ', 'DHAKA', 99444.0),
 ('Yoplait Original Yogurt Variety Pack ', 'KHULNA', 27876.0),
 ('Yoplait Original Yogurt Variety Pack ', 'RAJSHAHI', 30636.0),
 ('Yoplait Original Yogurt Variety Pack ', 'RANGPUR', 22332.0),
 ('Yoplait Original Yogurt Variety Pack ', 'SYLHET', 15264.0),
 ('Wonderful Pistachios (in shell)', 'BARISAL', 38082.0),
 ('Wonderful Pistachios (in shell)', 'CHITTAGONG', 96074.0),
 ('Wonderful Pistachios (in shell)', 'DHAKA', 196812.0

# Q10

In [18]:
cur = con.cursor()

cur.execute(''' SELECT i.supplier_,  td.month, avg(f.total_price)
                 FROM ecomdb.fact_table f 
                JOIN ecomdb.item_dim i on i.item_key= f.item_key 
                JOIN ecomdb.time_dim td on td.time_key = f.time_key
                group by i.supplier_, td.month
                order by i.supplier_, td.month asc ''')
cur.fetchall()


[('BIGSO AB', '1', 119.69589681440443),
 ('BIGSO AB', '10', 120.41189241346038),
 ('BIGSO AB', '11', 118.60735708165014),
 ('BIGSO AB', '12', 120.29326168337157),
 ('BIGSO AB', '2', 120.50479243786357),
 ('BIGSO AB', '3', 119.23176560840317),
 ('BIGSO AB', '4', 117.75505689001264),
 ('BIGSO AB', '5', 118.51862966484431),
 ('BIGSO AB', '6', 119.27643171806167),
 ('BIGSO AB', '7', 119.24802003261122),
 ('BIGSO AB', '8', 118.13369963369964),
 ('BIGSO AB', '9', 118.85876179831098),
 ('Bolsius Boxmeer', '1', 104.77929388188142),
 ('Bolsius Boxmeer', '10', 106.0727703757052),
 ('Bolsius Boxmeer', '11', 106.96655591775325),
 ('Bolsius Boxmeer', '12', 106.20986043689321),
 ('Bolsius Boxmeer', '2', 105.99348490892169),
 ('Bolsius Boxmeer', '3', 105.77266483516483),
 ('Bolsius Boxmeer', '4', 105.95010582669323),
 ('Bolsius Boxmeer', '5', 106.46366536689118),
 ('Bolsius Boxmeer', '6', 108.36575838414635),
 ('Bolsius Boxmeer', '7', 105.37763700648203),
 ('Bolsius Boxmeer', '8', 106.24988222824167)